In [1]:
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

from scipy.integrate import quad
import scipy.stats as stats

In [2]:
# Definición de variables

k = 8.617333262e-5 # eV / K
h = 4.135667e-15 # eV*s
h_bar = h/(2*np.pi) # eV*s
m =  0.5*510999 #eV  #0.5*9.1e-31 # kg
q = 1.602176e-19

Ec = -4.7
Ef = -5

L = 40e-9
W = 3*L

C_g = 0.1e-15
C_q = q*q*m*W*L/(2*np.pi*h_bar*h_bar)
C_es = C_g+C_q


# Parte I

In [13]:
# Funciones de integrales
def int_N(E,U,T,Vds):
    # print(E, (U+Ec)/(k*T))
    if E < Ef/(k*T):
        return 1/np.sqrt(E-(U+Ec)/(k*T))*(1/(1+np.exp(E-Ef/(k*T)))+1/(1+np.exp(E-(Ef-Vds)/(k*T))))
    else:
        return 1/np.sqrt(E-(U+Ec)/(k*T))*(np.exp(Ef/(k*T)-E)/(1+np.exp(Ef/(k*T)-E)) + np.exp((Ef-Vds)/(k*T)-E)/(1+np.exp((Ef-Vds)/(k*T)-E)))

def int_I(E,U,T, Vds):
    if E < Ef/(k*T):
        return np.sqrt(E-(U+Ec)/(k*T))*(1/(1+np.exp(E-Ef/(k*T)))-1/(1+np.exp(E-(Ef-Vds)/(k*T))))
    else:
        return np.sqrt(E-(U+Ec)/(k*T))*(np.exp(Ef/(k*T)-E)/(1+np.exp(Ef/(k*T)-E)) - np.exp((Ef-Vds)/(k*T)-E)/(1+np.exp((Ef-Vds)/(k*T)-E)))



# Funciones de cálculo
def get_N(U,T, Vds):
    cte = L*np.sqrt(k*T)*np.sqrt(2*m)/h
    integral = quad(int_N,
                    (U+Ec)/(k*T),
                    np.inf, 
                    args=(U, T, Vds))
    # print(cte, integral) 
    return cte*integral[0]

def get_U(N,N_0, V_gs):
    return q*q/C_es*(N-N_0)-V_gs*C_g/C_es

def get_I(U, T, Vds):
    cte = q*W*4*np.sqrt(2*m*k*T)*k*T/(h*h)
    integral = quad(int_I,
                    (U+Ec)/(k*T),
                    np.inf, 
                    args=(U, T, Vds))
    return cte*integral[0]

# Loop de autoconsistencia
def solve_I(Vds,T,V_gs, tol = 1e-4,lr = 1e-3):
    
    N_0 = get_N(0,T, 0)
    print(N_0)
    U_ = -1
    
    for _ in tqdm(range(int(1e4)), disable=False):
        N = get_N(U_,T, Vds)
        U = get_U(N,N_0,V_gs)

        if abs(U-U_) < tol:
            print('a')
            break

        else:
            U_ = U_ +lr*(U-U_)
    
    I = get_I(U,T,Vds)
    return U, I

U,I = solve_I(0.3, 
        1, 
        0.5, 
        lr = 0.01)

    

0.0


  0%|          | 0/10000 [00:00<?, ?it/s]C:\Users\benja\AppData\Local\Temp/ipykernel_13500/1917231119.py:5: RuntimeWarning: overflow encountered in exp
  return 1/np.sqrt(E-(U+Ec)/(k*T))*(1/(1+np.exp(E-Ef/(k*T)))+1/(1+np.exp(E-(Ef-Vds)/(k*T))))
C:\Users\benja\AppData\Local\Temp/ipykernel_13500/1917231119.py:20: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  integral = quad(int_N,
  9%|▊         | 858/10000 [00:14<02:39, 57.44it/s]

a



C:\Users\benja\AppData\Local\Temp/ipykernel_13500/1917231119.py:11: RuntimeWarning: overflow encountered in exp
  return np.sqrt(E-(U+Ec)/(k*T))*(1/(1+np.exp(E-Ef/(k*T)))-1/(1+np.exp(E-(Ef-Vds)/(k*T))))


In [14]:
U

-0.4481697804716027

In [15]:
I

122213.37502469096

# Parte II